![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# 6.01 Lab | Web Scraping Single Page

#### Business goal:

- Check the `case_study_gnod.md` file.
- Make sure you've understood the big picture of your project:

  - the goal of the company (`Gnod`),
  - their current product (`Gnoosic`),
  - their strategy, and
  - how your project fits into this context.

  Re-read the business case and the e-mail from the CTO, take a look at the flowchart and create an initial Trello board with the tasks you think you'll have to accomplish.

#### Instructions - Scraping popular songs

Your product will take a song as an input from the user and will output another song (the recommendation). In most cases, the recommended song will have to be similar to the inputted song, but the CTO thinks that if the song is on the top charts at the moment, the user will enjoy more a recommendation of a song that's also popular at the moment.

You have find data on the internet about currently popular songs. Billboard maintains a weekly Top 100 of "hot" songs here: [https://www.billboard.com/charts/hot-100](https://www.billboard.com/charts/hot-100).

It's a good place to start! Scrape the current top 100 songs and their respective artists, and put the information into a pandas dataframe.


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [243]:
url = "https://www.popvortex.com/music/charts/top-100-songs.php"

In [244]:
response = requests.get(url)
response.status_code

200

In [245]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
soup

In [366]:
artist = []
song = []
genre = []
year = []

num_iter = len("body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item")

songart = soup.select("body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item")
genlist = soup.select("body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item > div.chart-content")
yearlist = soup.select("body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item > div.chart-content > ul > li:nth-child(2)")

for i in range(num_iter):
    artist.append(songart[i].em.get_text())
    song.append(songart[i].cite.get_text())
    year.append(yearlist[i].get_text())    
    try:
        genre.append(genlist[i].ul.li.a.get_text())
    except:
        genre.append('Unknown')

    

In [397]:
top100 = pd.DataFrame({'artist':artist
                    ,'track':song
                    ,'genre':genre
                    ,'year':year})

In [398]:
top100['genre'].value_counts()


Pop              45
Country          20
New Release      17
Hip-Hop / Rap     6
Soundtrack        3
Rock              2
Unknown           2
Heavy Metal       1
R&B / Soul        1
Dance             1
Latin             1
Alternative       1
Name: genre, dtype: int64

In [399]:
top100

,artist,track,genre,year
0,Lizzo,About Damn Time,Pop,"Release Date: April 14, 2022"
1,Kate Bush,Running Up That Hill (A Deal with God),Pop,"Release Date: August 5, 1985"
2,Walker Hayes,Y'all Life,New Release,Genre: Country
3,Tom MacDonald,Names,New Release,Genre: Hip-Hop / Rap
4,Beyoncé,BREAK MY SOUL,Pop,"Release Date: June 21, 2022"
...,...,...,...,...
95,Dua Lipa,Levitating (feat. DaBaby),Pop,"Release Date: October 1, 2020"
96,Kenny Loggins,Danger Zone,Soundtrack,"Release Date: May 13, 1986"
97,GAYLE,abcdefu (feat. Royal & the Serpent),Pop,"Release Date: November 19, 2021"
98,Future,WAIT FOR U (feat. Drake & Tems),Hip-Hop / Rap,"Release Date: April 29, 2022"


In [400]:
def genresplit(value):
    if 'Genre:' in value:
        name = value.split('Genre: ')[1]
        
        return name
    else:
        return value

In [401]:
top100['genre'] = top100['genre'].apply(genresplit)
top100['year'] = top100['year'].apply(genresplit)

In [402]:
def tidy(value):

    type1 = value['genre']
    type2 = value['year']
    if str(type1) == 'New Release':
        return type2
    else:
        return type1

    
top100['genre'] = top100.apply(tidy,axis=1)

In [405]:
top100['year'] = top100['year'].str.split(", ", n = 1, expand = True)[1]
top100['year'] = top100['year'].fillna(2022)

In [420]:
top100

,artist,track,genre,year
0,Lizzo,About Damn Time,Pop,2022
1,Kate Bush,Running Up That Hill (A Deal with God),Pop,1985
2,Walker Hayes,Y'all Life,Country,2022
3,Tom MacDonald,Names,Hip-Hop / Rap,2022
4,Beyoncé,BREAK MY SOUL,Pop,2022
...,...,...,...,...
95,Dua Lipa,Levitating (feat. DaBaby),Pop,2020
96,Kenny Loggins,Danger Zone,Soundtrack,1986
97,GAYLE,abcdefu (feat. Royal & the Serpent),Pop,2021
98,Future,WAIT FOR U (feat. Drake & Tems),Hip-Hop / Rap,2022


![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# 6.02 Lab | Web Scraping Multiple Pages

#### Business goal:

- Check the `case_study_gnod.md` file.
- Make sure you've understood the big picture of your project:

  - the goal of the company (`Gnod`),
  - their current product (`Gnoosic`),
  - their strategy, and
  - how your project fits into this context.

  Re-read the business case and the e-mail from the CTO, take a look at the flowchart and create an initial Trello board with the tasks you think you'll have to accomplish.

#### Instructions 

#### Prioritize the MVP

In the previous lab, you had to scrape data about "hot songs". It's critical to be on track with that part, as it was part of the request from the CTO.

If you couldn't finish the first lab, use this time to go back there.

#### Expand the project

If you're done, you can try to expand the project on your own. Here are a few suggestions:

- Find other lists of hot songs on the internet and scrape them too: having a bigger pool of songs will be awesome!
- Apply the same logic to other "groups" of songs: the best songs from a decade or from a country / culture / language / genre.
- Wikipedia maintains a large collection of lists of songs: https://en.wikipedia.org/wiki/Lists_of_songs

#### Practice web scraping

As you've seen, scraping the internet is a skill that can get you all sorts of information. Here are some little challenges that you can try to gain more experience in the field:

- Retrieve an arbitrary Wikipedia page of "Python" and create a list of links on that page: `url ='https://en.wikipedia.org/wiki/Python'`
- Find the number of titles that have changed in the United States Code since its last release point: `url = 'http://uscode.house.gov/download/download.shtml'`
- Create a Python list with the top ten FBI's Most Wanted names: `url = 'https://www.fbi.gov/wanted/topten'`
- Display the 20 latest earthquakes info (date, time, latitude, longitude and region name) by the EMSC as a pandas dataframe: `url = 'https://www.emsc-csem.org/Earthquake/'`
- List all language names and number of related articles in the order they appear in [wikipedia.org](wikipedia.org): `url = 'https://www.wikipedia.org/'`
- A list with the different kind of datasets available in [data.gov.uk](data.gov.uk): `url = 'https://data.gov.uk/'`
- Display the top 10 languages by number of native speakers stored in a pandas dataframe: `url = 'https://en.wikipedia.org/wiki/List_of_languages_by_number_of_native_speakers'`